In [1]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Ren-CECps.csv to Ren-CECps (2).csv


In [ ]:
import pandas as pd
from sklearn import model_selection

labels = ['anger','anxiety','expect','hate','joy','love','sorrow','surprise','neutral']

train_data_path='Ren-CECps.csv'
train = pd.read_csv(train_data_path)

x_train, x_val = model_selection.train_test_split(
       train,
       test_size=0.1,
       random_state=123,
       stratify=pd.factorize(train.polarity)[0])

cols = labels
cols.append('message')
x_train = x_train[cols]
x_train.rename(columns={'message':'text'}, inplace = True)

x_val = x_val[cols]
x_val.rename(columns={'message':'text'}, inplace = True)

x_train

,anger,anxiety,expect,hate,joy,love,sorrow,surprise,neutral,text
9499,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0,像甘露，甜甜蜜蜜，清爽可口。
28588,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0,他们的教学水平不见得比今天这些师范类院校毕业的教师差，某些方面甚至要比这些科班的老师要强。
27483,0.0,0.4,0.0,0.0,0.0,0.0,0.6,0.0,0,到现在我也没有吃到那家店的点心，因为那年的秋天，外婆就被查出了肠癌，无法再照顾我了，我也就此...
32565,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0,读研时，有事没事地总爱往我们外语学院的资料室跑。
46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,“旧账”所对应者，乃“新账”也。
...,...,...,...,...,...,...,...,...,...,...
11427,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0,大正月天，说人家是假钞？
33439,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0,唐代诗人岑参曾在此建造别墅，勤于耕读，和高冠瀑布结下不解之缘。
20216,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0,窗外雪花漫天飞舞，遮不住你我沉重又甜蜜的牵念。
22960,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0,我哀，辗转中能否如你般淡薄？


In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
# !pip install transformers
from transformers import AutoTokenizer,AutoModelForSequenceClassification

In [ ]:
# !pip install transformers
# !pip install datasets
# !pip install -U sentence-transformers
# !pip install pytorch-pretrained-bert pytorch-ignite ipdb
# !pip install transformers-interpret
# !!wget https://github.com/chrisdonahue/ilm/archive/master.zip
# !!unzip master.zip &> /dev/null
# %cd ./ilm-master/
# !pip install -e .
import sys, os
sys.path.append(os.getcwd())
%cd ../

/


In [ ]:
import torch

In [ ]:
# Prepare Model from hugging face: https://huggingface.co/hfl/chinese-roberta-wwm-ext
MODEL_NAME = 'hfl/chinese-roberta-wwm-ext'
labels = ['anger','anxiety','expect','hate','joy','love','sorrow','surprise','neutral']
NUM_LABELS =  len(labels)

# load a pre-trained tokenizer for a specific model architecture
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Prepare your dataset by feeding each sentence to your tokenizer object. You can apply these preprocessing steps to all the splits of our dataset at once by using the map method:
def tokenize_function(examples):
  return tokenizer(examples["text"].to_list(), padding="max_length", max_length=256, truncation=True)

train_encodings = tokenize_function(x_train)
val_encodings = tokenize_function(x_val)

# Converting a DataFrame to a dictionary through record: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html
train_labels = [[example[label] for label in labels] for example in x_train.to_dict('records')]
val_labels = [[example[label] for label in labels] for example in x_val.to_dict('records')]

class Dataset(torch.utils.data.Dataset):
    # For each new Dataset, initialize (that is, specify) it with a encoding (word vector) and a label (emotoinal label)
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    # loops through each key (representing data fields) in self.encodings; For each key, it extracts the value corresponding to the current index idx and converts it to a PyTorch tensor using torch.tensor.
      #example original text when idx=0: '像甘露，甜甜蜜蜜，清爽可口。'
      #example encoding: 'input_ids': [101, 1008, 4491, 7463, 8024, 4494, 4494, 6057, 6057, 8024, 3926, 4272, 1377, 1366, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
      # example item['labels']: add key labels of current idx as [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7000, 0.0000, 0.0000, 0.0000]
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
eval_dataset = Dataset(val_encodings, val_labels)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Create a custom model to handle multilabel for each instance
class BertForMultilabelSequenceClassification(BertForSequenceClassification):
  # calls the constructor of the base class (BertForSequenceClassification) using super().__init__(config).
      # This ensures that the base class is properly initialized with the provided in config
    def __init__(self, config):
        super().__init__(config)

    # Data/config to model
    def forward(self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict # if return dictionary not provided, use default in the configuration
        # BERT Model Execution
        outputs = self.bert(input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict)
        # Post-processing and Classification
        pooled_output = outputs[1] # Pooled output, corresponding to the [CLS] (classification) token's embedding
        pooled_output = self.dropout(pooled_output) # Randonly zero out 10% of the values to regulize (hidden_dropout_prob=0.1)
        logits = self.classifier(pooled_output) # Pass through the classifier, returns unnormalized scores for each class (e.g., torch.tensor([1.5, -0.5,..., 2.0]))

        loss = None
        if labels is not None:
            loss_fct = torch.nn.BCEWithLogitsLoss() #  Compute the loss for a multi-label classification task: Binary Cross-Entropy with Logits Loss; a loss function that combines a Sigmoid layer and the Binary Cross-Entropy Loss in one single class
            # compare predicted logits with actual label
            loss = loss_fct(logits.view(-1, self.num_labels), # view: reshapes the tensors to be of shape (number of instance(posts), number of labels)
                            labels.float().view(-1, self.num_labels))

        if not return_dict:
            output = (logits,) + outputs[2:] #  raw prediction + outputs excluding token-level output and pooled output
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states, # (batch_size, sequence_length, hidden_size)
            attentions=outputs.attentions) # (batch_size, num_heads, sequence_length, sequence_length)

In [ ]:
import accelerate

In [ ]:
# choose GPU accelerator in the `Runtime` menu
!nvidia-smi

Sat Jun 15 07:10:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# pip install accelerate -U

In [ ]:
# Instantiate a TrainingArguments
  # Contains all the hyperparameters we can tune for the Trainer
from transformers import TrainingArguments
from transformers import Trainer

# Uuse defaults, provide is a directory to save checkpoints
training_args = TrainingArguments("./")
print(training_args)
# Customize hyperparameters to tune the Trainer
training_args = TrainingArguments(
    output_dir="./",
    per_device_train_batch_size=32,  # batch size (number of samples) per device (GPU) during training, normally 16 or 32 is recommended (see Devlin et al. 2018)
    per_device_eval_batch_size=32,   # batch size for evaluation, normally set to be the same as traning batch size
    learning_rate = 2e-5,            # step size at each iteration, 1e−5, 2e−5, 3e−5, and 5e-5 are commonly recommended for fine-tuning transformer
    num_train_epochs = 4,            # total number of training epochs; 1 epoch means that every sample in the training dataset has been seen once
    logging_steps=100,               # every 100 steps (batchs), the model will log loss metric
    eval_steps=100,                  # every 100 steps (batchs), the model will log evaluation metric
    evaluation_strategy='steps'      # evaluation will be run at regular intervals defined by eval_steps
)


# Define the model:
model = BertForMultilabelSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))

def accuracy_thresh(y_pred, y_true, thresh=0.5, sigmoid=True):
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid:
        y_pred = y_pred.sigmoid() # convert the logits into probabilities
    return ((y_pred>thresh)==(y_true>thresh).bool()).float().mean().item()

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    acc = accuracy_thresh(predictions, labels)
    return {'accuracy_thresh': acc}

# instantiate a Trainer
trainer = Trainer(
    model=model,                         # instantiated Hugging Face Transformers model to be trained
    args=training_args,                  # defined training arguments
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,           # evaluation dataset
    compute_metrics=compute_metrics      # metrics for evaluation
)

# Tell pytorch to run this model on the GPU.
model.cuda()

# Fine-tune our model
trainer.train()

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=None,
eval_strategy=no,
evaluation_strategy=None,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_c

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForMultilabelSequenceClassification were not initialized from the model checkpoint at hfl/chinese-roberta-wwm-ext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy Thresh
100,0.335400,0.276343,0.888601
200,0.262700,0.247527,0.900201
300,0.246500,0.236785,0.907168
400,0.238900,0.231275,0.912377
500,0.233600,0.227907,0.913687
600,0.231500,0.225467,0.912632
700,0.226500,0.224861,0.915828
800,0.226700,0.221213,0.916755
900,0.225700,0.219670,0.912121
1000,0.222000,0.218462,0.915636


TrainOutput(global_step=3912, training_loss=0.20295093322824115, metrics={'train_runtime': 6651.7095, 'train_samples_per_second': 18.817, 'train_steps_per_second': 0.588, 'total_flos': 1.6467577213722624e+16, 'train_loss': 0.20295093322824115, 'epoch': 4.0})

In [ ]:
!pwd

/


In [ ]:
# Mount Google Drive in Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Save the trained model
trainer.save_model("/content/drive/MyDrive/model_output")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/model_output")

# Save the configuration
model.config.save_pretrained("/content/drive/MyDrive/model_output")

In [ ]:
print(model.config)

BertConfig {
  "_name_or_path": "hfl/chinese-roberta-wwm-ext",
  "architectures": [
    "BertForMultilabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "directionality": "bidi",
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,


In [ ]:
# check pretrained model parameters
check = BertConfig.from_pretrained("hfl/chinese-roberta-wwm-ext")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

In [ ]:
print(check)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "directionality": "bidi",
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

